In [2]:
import pandas as pd
import os,glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *
import re

In [3]:
df = unpkl('raw.pkl')
lf = unpkl('lraw.pkl')

# Overall standard deviation

In [17]:
allmods = [item for sublist in df.thickness for item in sublist]
alltrads = [item for sublist in lf.thickness for item in sublist]
allthicknesses = allmods + alltrads

In [18]:
import numpy as np

In [19]:
thickstd = np.std(allthicknesses)
thickstd

0.08280782637407

# Modern dupes

### Enumeration of Cases

In [22]:
from itertools import combinations

In [23]:
moddupes = df.catalog.value_counts().loc[lambda x: x > 1]
for moddupe in moddupes.index:
    tmp = df[df.catalog==moddupe]
    
    # we exclude case where both Leah and I measured binder 4
    tmp = tmp.sort_values('fname')
    fname_string = '|'.join(list(tmp.fname))
    
    # we exclude case where `catalog_manbran` is the same and the medians are within 1SD
    l = list(tmp.thickness.apply(np.median))
    medpairs = list(combinations(l, 2))
    meddiffs = [abs(item[0]-item[1]) for item in medpairs]
    
    if 'thickness_dc_binder4.xlsx|thickness_ll_binder4' in fname_string: # hers are spread across multiple files
        continue
    elif all([diff < thickstd for diff in meddiffs]) and len(tmp.catalog_manbran.unique())==1:
        continue
    else:
        print(moddupe,end='\t')

5606kk	2559	2523	5150	3131	4792e	4792n	27	2166	5408f	5408g	5408h	5408i	5408j	1433	5408c	5408d	5408b	5154	40	72	0016	2304	2077h	2077j	2512	5408a	2372dd	5036	5155	5408e	

In [24]:
df[df.catalog=='5155']

,catalog,thickness,thickness_str,fname,mera,catalog_manbran
669,5155,"[0.163, 0.165, 0.165]","[0.163, 0.165, 0.165]",thickness_ac_binder2_6.xlsx,mod,5155_ek
670,5155,"[0.166, 0.171, 0.168]","[0.166, 0.171, 0.168]",thickness_ac_binder2_6.xlsx,mod,5155_eg


In [25]:
lf[lf.catalog=='5155']

,catalog,thickness,thickness_str,fname,mera
